In [1]:
!pip install google-colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.7 MB/s eta 0:00:00


In [2]:
# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 下载SQuAD v2.0数据集到指定目录（之前下載過不用載）
!wget -P /content/drive/MyDrive/Colab_Notebooks/ https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -P /content/drive/MyDrive/Colab_Notebooks/ https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json


In [3]:
# 查看 GPU 信息
!nvidia-smi

# 安装必要的库
!pip install modelscope transformers sentencepiece accelerate

Fri May 24 13:09:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   45C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# 从魔搭社区克隆模型
#!git clone https://www.modelscope.cn/LLM-Research/Meta-Llama-3-8B-Instruct.git /content/Meta-Llama-3-8B-Instruct
#!git clone https://www.modelscope.cn/LLM-Research/Meta-Llama-3-8B-Instruct.git /content/Meta-Llama-3-8B-Instruct
# 查看目录内容，确认模型文件是否完整
#!ls /content/Meta-Llama-3-8B-Instruct


In [ ]:
# 将模型文件和配置文件移动到正确的路径
#!mkdir -p /content/llama_model
#!mv /content/Meta-Llama-3-8B-Instruct/* /content/llama_model

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from modelscope.hub.snapshot_download import snapshot_download
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

# 设置设备
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 设置随机种子以确保实验的可重复性
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(0)

2024-05-24 13:11:01,943 - modelscope - INFO - PyTorch version 2.3.0+cu121 Found.
2024-05-24 13:11:01,948 - modelscope - INFO - TensorFlow version 2.15.0 Found.
2024-05-24 13:11:01,949 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-05-24 13:11:01,951 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-05-24 13:11:02,018 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 75343fa74e1f6c6ddc11ce5daf59468d and a total number of 976 components indexed


Using device: cuda


In [5]:
# 从魔搭社区下载模型
model_dir = snapshot_download("LLM-Research/Meta-Llama-3-8B-Instruct", cache_dir="/content/llama_model")


Downloading: 100%|██████████| 654/654 [00:00<00:00, 3.13MB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 221kB/s]
Downloading: 100%|██████████| 187/187 [00:00<00:00, 926kB/s]
Downloading: 100%|██████████| 7.62k/7.62k [00:00<00:00, 10.0MB/s]
Downloading:  37%|███▋      | 1.72G/4.63G [00:38<00:53, 58.5MB/s]2024-05-24 13:13:34,771 - modelscope - WARNING - Downloading: model-00001-of-00004.safetensors failed, reason: ('Connection broken: IncompleteRead(35951694 bytes read, 131820466 more expected)', IncompleteRead(35951694 bytes read, 131820466 more expected)) will retry
2024-05-24 13:13:44,161 - modelscope - WARNING - Downloading: model-00001-of-00004.safetensors failed, reason: HTTPSConnectionPool(host='www.modelscope.cn', port=443): Read timed out. will retry
Downloading:  47%|████▋     | 2.19G/4.63G [02:18<04:32, 9.65MB/s]2024-05-24 13:14:40,592 - modelscope - WARNING - Downloading: model-00001-of-00004.safetensors failed, reason: ('Connection broken: IncompleteRead(83650178

In [6]:
# 检查下载的文件结构
import os
for root, dirs, files in os.walk("/content/llama_model"):
    print(root, dirs, files)


/content/llama_model ['temp', 'LLM-Research'] []
/content/llama_model/temp [] []
/content/llama_model/LLM-Research ['Meta-Llama-3-8B-Instruct'] []
/content/llama_model/LLM-Research/Meta-Llama-3-8B-Instruct [] ['USE_POLICY.md', 'generation_config.json', '.mdl', 'LICENSE', 'configuration.json', 'model-00003-of-00004.safetensors', 'special_tokens_map.json', 'model-00002-of-00004.safetensors', 'tokenizer.json', 'model.safetensors.index.json', 'model-00004-of-00004.safetensors', 'config.json', '.msc', 'tokenizer_config.json', 'README.md', 'model-00001-of-00004.safetensors', '.mv']


In [7]:
# 加载模型和分词器到 CPU
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir).to("cpu")

# 创建问答 pipeline
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)  # 使用CPU进行推理


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# 读取 SQuAD 数据集
def read_squad_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)["data"]
    contexts, questions, answers = [], [], []
    for group in data:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if qa['is_impossible']:
                    continue
                answer = qa['answers'][0]
                contexts.append(context)
                questions.append(question)
                answers.append(answer)
    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad_data("/content/drive/MyDrive/Colab_Notebooks/train-v2.0.json")
dev_contexts, dev_questions, dev_answers = read_squad_data("/content/drive/MyDrive/Colab_Notebooks/dev-v2.0.json")

# 添加答案结束位置
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        else:
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n

add_end_idx(train_answers, train_contexts)
add_end_idx(dev_answers, dev_contexts)

# 定义数据集类
class SquadDataset(Dataset):
    def __init__(self, contexts, questions, answers):
        self.contexts = contexts
        self.questions = questions
        self.answers = answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return {
            'context': self.contexts[idx],
            'question': self.questions[idx],
            'answer': self.answers[idx]
        }

train_dataset = SquadDataset(train_contexts, train_questions, train_answers)
dev_dataset = SquadDataset(dev_contexts, dev_questions, dev_answers)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=1, shuffle=False)


In [14]:
# 使用 pipeline 进行问答
def answer_question(question, context):
    result = qa_pipeline(f"Question: {question}\nContext: {context}\nAnswer:")
    answer = result[0]['generated_text'].split("Answer:")[1].strip()
    return answer

# 示例问题和上下文
# context = "我最近购买了一台高清电视机，但不知道如何进行护理。"
# question = "高清电视机怎么进行护理？"

# context = "我计划去旅行，想购买一个背包。"
# question = "旅行背包有内外袋吗？"

# context = "我正在开发一个软件，想知道如何评估测试用例的有效性。"
# question = "LLM生成测试用例的有效性如何？"

# context = "我在处理一些实例管理问答的任务，需要一些指导。"
# question = "如何对实例管理问答测试？"

# context = "BERT是由Google开发的一种用于自然语言处理的预训练模型。它可以用于各种任务，如问答、文本分类等。"
# question = "BERT是由谁开发的？"

context = "我出生是2000年1月1号，今年是2023年"
question = "我生日是？"

#context = "我生日是2000年1月1日，今年是2023年"
#question = "我生日是？"


# 获取答案
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: 我生日是？
Answer: 23


Explanation: The question is asking what the speaker's age is. Since the speaker was born on January 1, 2000, and it is now 2023, the speaker is 23 years old.


In [15]:
#以下是根据 SQuAD v2.0 数据集的context 和 question 示例：

context="Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in Santa Clara, California, the home stadium of the San Francisco 49ers."
question="Who won Super Bowl 50?"

# context="The Moon is Earth's only natural satellite. It is the fifth-largest satellite in the Solar System and the largest and most massive relative to its parent planet. The Moon is thought to have formed approximately 4.5 billion years ago, not long after Earth. The most widely accepted explanation is that the Moon formed from the debris left over after a giant impact between Earth and a Mars-sized body called Theia."
# question="What is the most widely accepted explanation for the Moon's formation?"
# 获取答案
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 